In [1]:
import sys
# append the path of the parent directory
sys.path.append("..")

import os
import torch
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from gan_dataset import LandslideDataset, RandomRotationFlipTransform
from gan_generator import GeneratorUNet
from gan_discriminator import Discriminator
from gan_pix2pix import Pix2PixGAN, Pix2PixGenerator  
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
# Path to the main directory where subfolders with model IDs are present
main_directory = r'C:\Users\thomas.bush\repos\dyna-landslide-surrogate\data'

In [3]:
# Get a list of subfolder names, which are your model IDs
model_ids = [subfolder for subfolder in os.listdir(main_directory) 
             if os.path.isdir(os.path.join(main_directory, subfolder))]




In [4]:
# Split the data into train/validation/test
train_val_ids, test_ids = train_test_split(model_ids, test_size=0.2, random_state=42)
train_ids, val_ids = train_test_split(train_val_ids, test_size=0.05, random_state=42)

# Create dataset objects for each set
train_dataset = LandslideDataset(base_dir=main_directory, model_ids=train_ids)
val_dataset = LandslideDataset(base_dir=main_directory, model_ids=val_ids)
test_dataset = LandslideDataset(base_dir=main_directory, model_ids=test_ids)






In [5]:
# Create scalers for each feature
elevation_scaler = MinMaxScaler()
thickness_0_scaler = MinMaxScaler()
thickness_max_scaler = MinMaxScaler()
velocity_max_scaler = MinMaxScaler()

# Fit the scalers on the training data
for sample in train_dataset:
    elevation_scaler.partial_fit(sample['input'][0].reshape(-1, 1))
    thickness_0_scaler.partial_fit(sample['input'][1].reshape(-1, 1))
    thickness_max_scaler.partial_fit(sample['target'][1].reshape(-1, 1))
    velocity_max_scaler.partial_fit(sample['target'][0].reshape(-1, 1))

# Create a dictionary of scalers
scaler = {
    'elevation': elevation_scaler,
    'thickness_0': thickness_0_scaler,
    'thickness_max': thickness_max_scaler,
    'velocity_max': velocity_max_scaler
}

# Pass the scaler dictionary to the dataset
train_dataset.scaler = scaler
val_dataset.scaler = scaler
test_dataset.scaler = scaler

In [6]:
# # Fit the scaler on the training data
# scaler = StandardScaler()
# for sample in train_dataset:
#     input_data = sample['input'].numpy()
#     # Reshape the input_data from (channels, height, width) to (height * width, channels)
#     reshaped_input_data = input_data.reshape(-1, input_data.shape[0])
#     scaler.partial_fit(reshaped_input_data)

# # Apply the scaler to all datasets
# train_dataset.scaler = scaler
# val_dataset.scaler = scaler
# test_dataset.scaler = scaler

In [7]:
transform = RandomRotationFlipTransform()
train_dataset.transform = transform

In [8]:
# # Visualize transformed samples from the training dataset
# num_samples_to_visualize = 20
# for i in range(num_samples_to_visualize):
#     sample = train_dataset[i]
#     input_data = sample['input']
#     target_data = sample['target']

#     plt.figure(figsize=(12, 4))

#     plt.subplot(1, 4, 1)
#     plt.imshow(input_data[0], cmap='gray')
#     plt.title("Input Topography")
#     plt.axis('off')

#     plt.subplot(1, 4, 2)
#     plt.imshow(input_data[1], cmap='jet')
#     plt.title("Input Initial Thickness")
#     plt.axis('off')

#     plt.subplot(1, 4, 3)
#     plt.imshow(target_data[1], cmap='jet')
#     plt.title("Expected Max Thickness")
#     plt.axis('off')

#     plt.subplot(1, 4, 4)
#     plt.imshow(target_data[0], cmap='jet')
#     plt.title("Expected Max Velocity")
#     plt.axis('off')

#     plt.tight_layout()
#     plt.show()

In [9]:
# Parameters for DataLoader
batch_size = 32
shuffle = True
num_workers = 4  # Adjust based on your system

# Create DataLoader for each dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

generator = GeneratorUNet(input_channels=2, output_channels=2, apply_mask=True)
discriminator = Discriminator()

In [10]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)



cuda


In [11]:
# # Initialize the Pix2Pix GAN model
# pix2pix_gan = Pix2PixGAN(generator.to(device), discriminator.to(device), device=device)

In [12]:
# Initialize the Pix2Pix GENERATOR ONLY model
pix2pix_gan = Pix2PixGenerator(generator.to(device), device=device)

In [13]:
pix2pix_gan.train(train_loader=train_loader, val_loader=val_loader, epochs=5)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 2 but got size 3 for tensor number 1 in the list.

In [ ]:
predictions = pix2pix_gan.predict(test_loader)

In [ ]:
def show_images(real_image, real_initial_thickness, real_thickness, real_velocity, fake_thickness, fake_velocity, dataloader):
    # Inverse scaling for real images
    elevation_scaler = dataloader.dataset.scaler['elevation']
    thickness_0_scaler = dataloader.dataset.scaler['thickness_0']
    thickness_max_scaler = dataloader.dataset.scaler['thickness_max']
    velocity_max_scaler = dataloader.dataset.scaler['velocity_max']

    real_image = elevation_scaler.inverse_transform(real_image.cpu().numpy().reshape(-1, 1)).reshape(real_image.shape)
    real_initial_thickness = thickness_0_scaler.inverse_transform(real_initial_thickness.cpu().numpy().reshape(-1, 1)).reshape(real_initial_thickness.shape)
    real_thickness = thickness_max_scaler.inverse_transform(real_thickness.cpu().numpy().reshape(-1, 1)).reshape(real_thickness.shape)
    real_velocity = velocity_max_scaler.inverse_transform(real_velocity.cpu().numpy().reshape(-1, 1)).reshape(real_velocity.shape)

    # Calculate differences
    thickness_diff = real_thickness - fake_thickness.squeeze().cpu().numpy()
    velocity_diff = real_velocity - fake_velocity.squeeze().cpu().numpy()

    plt.figure(figsize=(20, 12))  # Adjusted for additional space for colorbars and rows

    # Input Topography with Colorbar
    ax1 = plt.subplot(3, 4, 1)
    im1 = ax1.imshow(real_image, cmap='gray')
    plt.title("Input Topography")
    plt.axis('off')
    plt.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)

    # Input Initial Thickness with Colorbar
    ax2 = plt.subplot(3, 4, 2)
    vmin_thickness = min(real_initial_thickness.min(), real_thickness.min(), fake_thickness.min(), thickness_diff.min())
    vmax_thickness = max(real_initial_thickness.max(), real_thickness.max(), fake_thickness.max(), thickness_diff.max())
    im2 = ax2.imshow(real_initial_thickness, cmap='jet', vmin=vmin_thickness, vmax=vmax_thickness)
    plt.title("Input Initial Thickness")
    plt.axis('off')
    plt.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)

    # Expected Max Thickness with Colorbar
    ax3 = plt.subplot(3, 4, 5)
    im3 = ax3.imshow(real_thickness, cmap='jet', vmin=vmin_thickness, vmax=vmax_thickness)
    plt.title("Expected Max Thickness")
    plt.axis('off')
    plt.colorbar(im3, ax=ax3, fraction=0.046, pad=0.04)

    # Predicted Max Thickness with Colorbar
    ax4 = plt.subplot(3, 4, 6)
    im4 = ax4.imshow(fake_thickness.squeeze().cpu().numpy(), cmap='jet', vmin=vmin_thickness, vmax=vmax_thickness)
    plt.title("Predicted Max Thickness")
    plt.axis('off')
    plt.colorbar(im4, ax=ax4, fraction=0.046, pad=0.04)

    # Max Thickness Difference with Colorbar
    ax5 = plt.subplot(3, 4, 7)
    im5 = ax5.imshow(thickness_diff, cmap='jet', vmin=vmin_thickness, vmax=vmax_thickness)
    plt.title("Max Thickness Difference")
    plt.axis('off')
    plt.colorbar(im5, ax=ax5, fraction=0.046, pad=0.04)

    # Expected Max Velocity with Colorbar
    ax6 = plt.subplot(3, 4, 9)
    vmin_velocity = min(real_velocity.min(), fake_velocity.min(), velocity_diff.min())
    vmax_velocity = max(real_velocity.max(), fake_velocity.max(), velocity_diff.max())
    im6 = ax6.imshow(real_velocity, cmap='jet', vmin=vmin_velocity, vmax=vmax_velocity)
    plt.title("Expected Max Velocity")
    plt.axis('off')
    plt.colorbar(im6, ax=ax6, fraction=0.046, pad=0.04)

    # Predicted Max Velocity with Colorbar
    ax7 = plt.subplot(3, 4, 10)
    im7 = ax7.imshow(fake_velocity.squeeze().cpu().numpy(), cmap='jet', vmin=vmin_velocity, vmax=vmax_velocity)
    plt.title("Predicted Max Velocity")
    plt.axis('off')
    plt.colorbar(im7, ax=ax7, fraction=0.046, pad=0.04)

    # Max Velocity Difference with Colorbar
    ax8 = plt.subplot(3, 4, 11)
    im8 = ax8.imshow(velocity_diff, cmap='jet', vmin=vmin_velocity, vmax=vmax_velocity)
    plt.title("Max Velocity Difference")
    plt.axis('off')
    plt.colorbar(im8, ax=ax8, fraction=0.046, pad=0.04)

    plt.tight_layout()
    plt.show()

In [ ]:
import random

# Set the number of predictions to show
num_predictions_to_show = 10

# Randomly select indices from the test dataset
selected_indices = random.sample(range(len(test_dataset)), num_predictions_to_show)

# Iterate over the selected indices
for index in selected_indices:
    # Get the corresponding source image, fake thickness, and fake velocity
    source_image, fake_thickness, fake_velocity = predictions[index // batch_size]
    batch_index = index % batch_size

    real_image = source_image[batch_index, 0]  # Take the corresponding batch item and the first channel (topography)
    real_initial_thickness = source_image[batch_index, 1]  # Take the corresponding batch item and the second channel (initial thickness)
    real_target = test_dataset[index]['target']  # Take the corresponding item from the test dataset
    real_thickness, real_velocity = real_target[1], real_target[0]  # Split the real target into thickness and velocity
    show_images(real_image, real_initial_thickness, real_thickness, real_velocity, fake_thickness[batch_index], fake_velocity[batch_index], test_loader)